# Predicting Heart Stroke

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.neural_network import MLPClassifier
#from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve
from sklearn.metrics import auc, roc_auc_score, precision_score, recall_score
from sklearn.metrics import classification_report

In [3]:
train=pd.read_csv(r'C:\Users\matan\Downloads\healthcare-dataset-stroke-data\\train_2v.csv')
test=pd.read_csv(r'C:\Users\matan\Downloads\healthcare-dataset-stroke-data\\test_2v.csv')
train.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [4]:
test.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,14152,Female,14.0,0,0,No,children,Rural,95.16,21.2,NaN
3,12997,Male,28.0,0,0,No,Private,Urban,94.76,23.4,NaN
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked


In [7]:
train.shape
#test.shape
#print('Train Data:'.format(train.shape))
#print('Test Data:'.format(test.shape))

(43400, 12)

In [8]:
test.shape

(18601, 11)

In [9]:
train_missing_values=train.isnull().sum()
train_missing_values

id                       0
gender                   0
age                      0
hypertension             0
heart_disease            0
ever_married             0
work_type                0
Residence_type           0
avg_glucose_level        0
bmi                   1462
smoking_status       13292
stroke                   0
dtype: int64

In [10]:
test_missing_values=test.isnull().sum()
test_missing_values

id                      0
gender                  0
age                     0
hypertension            0
heart_disease           0
ever_married            0
work_type               0
Residence_type          0
avg_glucose_level       0
bmi                   591
smoking_status       5751
dtype: int64

In [11]:
train_data=train.dropna(axis=0,how='any')
test_data=test.dropna(axis=0,how='any')
print('Train Data Shape: {}'.format(train_data.shape))
print('Test Data Shape: {}'.format(test_data.shape))

Train Data Shape: (29072, 12)
Test Data Shape: (12423, 11)


In [12]:
train_data["stroke"].unique()

array([0, 1], dtype=int64)

In [13]:
train_data["smoking_status"].unique()

array(['never smoked', 'formerly smoked', 'smokes'], dtype=object)

In [14]:
train_data["stroke"].value_counts()

0    28524
1      548
Name: stroke, dtype: int64

In [15]:
train_data["gender"].value_counts()

Female    17852
Male      11213
Other         7
Name: gender, dtype: int64

In [16]:
train_data.groupby(["gender"])["stroke"].value_counts()

gender  stroke
Female  0         17539
        1           313
Male    0         10978
        1           235
Other   0             7
Name: stroke, dtype: int64

In [17]:
train_data.groupby(["smoking_status"])["stroke"].value_counts()

smoking_status   stroke
formerly smoked  0          6919
                 1           180
never smoked     0         15491
                 1           256
smokes           0          6114
                 1           112
Name: stroke, dtype: int64

In [24]:
str_data=train_data.select_dtypes(include=['object'])
str_dt=test_data.select_dtypes(include=['object'])

In [25]:
int_data=train_data.select_dtypes(include=['integer',"float"])
int_dt=test_data.select_dtypes(include=['integer',"float"])

In [26]:
label=LabelEncoder()
features=str_data.apply(label.fit_transform)
features=features.join(int_data)
features.head()

,gender,ever_married,work_type,Residence_type,smoking_status,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
1,1,1,2,1,1,30468,58.0,1,0,87.96,39.2,0
3,0,1,2,0,0,56543,70.0,0,0,69.04,35.9,0
6,0,1,2,1,0,52800,52.0,0,0,77.59,17.7,0
7,0,1,3,0,1,41413,75.0,0,1,243.53,27.0,0
8,0,1,2,0,2,15266,32.0,0,0,77.67,32.3,0


In [27]:
test1=str_dt.apply(label.fit_transform)
Test=test1.join(int_dt)
Test.head()

,gender,ever_married,work_type,Residence_type,smoking_status,id,age,hypertension,heart_disease,avg_glucose_level,bmi
0,1,1,2,1,0,36306,80.0,0,0,83.84,21.1
1,0,1,3,0,0,61829,74.0,0,1,179.50,26.0
4,0,1,0,0,1,40801,63.0,0,0,83.57,27.6
5,0,1,2,1,1,9348,66.0,1,0,219.98,32.2
7,1,1,0,1,1,60512,46.0,0,0,120.80,32.5


In [33]:
xtrain=features.drop(["stroke","id"],axis=1)
xtrain.head()

,gender,ever_married,work_type,Residence_type,smoking_status,age,hypertension,heart_disease,avg_glucose_level,bmi
1,1,1,2,1,1,58.0,1,0,87.96,39.2
3,0,1,2,0,0,70.0,0,0,69.04,35.9
6,0,1,2,1,0,52.0,0,0,77.59,17.7
7,0,1,3,0,1,75.0,0,1,243.53,27.0
8,0,1,2,0,2,32.0,0,0,77.67,32.3


In [34]:
ytrain=features["stroke"]
ytrain.head()

1    0
3    0
6    0
7    0
8    0
Name: stroke, dtype: int64

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(xtrain,ytrain)

In [37]:
model=GaussianNB()
model.fit(x_train,y_train)

GaussianNB(priors=None)

In [39]:
predict=model.predict(x_test)
predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [40]:
test_score=model.score(x_test,y_test)
print("NBtest_score:",test_score)


NBtest_score: 0.926114474408


In [41]:
train_score=model.score(x_train,y_train)
print("NBtest_score:",train_score)

NBtest_score: 0.924784443221


In [42]:
from sklearn.model_selection import cross_validate
cv_results=cross_validate(model,xtrain,ytrain,cv=5)
cv_results

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 0.02099967,  0.01999927,  0.01800036,  0.01799917,  0.01799965]),
 'score_time': array([ 0.00399971,  0.00300074,  0.00299978,  0.00300026,  0.0030005 ]),
 'test_score': array([ 0.92605331,  0.91711092,  0.92364574,  0.92741658,  0.92706004]),
 'train_score': array([ 0.92496883,  0.92587178,  0.92466784,  0.92351019,  0.9233845 ])}

In [43]:
nb_conf_mtr=pd.crosstab(y_test,predict)
nb_conf_mtr

col_0,0,1
stroke,,
0,6695,429
1,108,36


In [44]:
nbreport=classification_report(y_test,predict)
print(nbreport)

             precision    recall  f1-score   support

          0       0.98      0.94      0.96      7124
          1       0.08      0.25      0.12       144

avg / total       0.97      0.93      0.94      7268



In [45]:
dt_mod=DecisionTreeClassifier(criterion='entropy',max_depth=8)
dt_mod.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [46]:
ts_dt_score=dt_mod.score(x_test,y_test)
print("Dtest_score:",ts_dt_score)

Dtest_score: 0.977710511833
